# Thimble 3: The All-Bfloat16 Revue 🎭

**Starring:** Quantization at every step!

**Objective:** Train in bfloat16 and record **everything** in bfloat16. No dtype conversions. What PyTorch does, we do.

## What We Learned

**Thimble 1:** Trained in bfloat16, but saved optimizer states as float32 (converting during recording)

**Thimble 2:** Trained in float32, proved the AdamW formula works (max_diff < 3e-08)

**The Problem:** Thimble 1's saved states are MORE PRECISE than what PyTorch actually used during training. We converted bf16→f32 when saving, giving us higher precision states than the actual training.

**The Solution:** Record states in bfloat16. Match PyTorch's actual precision, quantization and all.

## The All-Bfloat16 Pipeline

PyTorch does this (when param is bfloat16):
```python
# All in bfloat16
m[t] = (0.9 * m[t-1] + 0.1 * grad[t]).bfloat16()
v[t] = (0.999 * v[t-1] + 0.001 * grad[t]²).bfloat16()
m_hat = (m[t] / bias_corr1).bfloat16()
v_hat = (v[t] / bias_corr2).bfloat16()
dW = (-lr * m_hat / sqrt(v_hat)).bfloat16()
W[t] = (W[t-1] + dW).bfloat16()
```

Quantization at every intermediate step. This is what we'll simulate.

## Parameters

In [1]:
# Model architecture
VOCAB_SIZE = 10000
HIDDEN_DIM = 64
N_LAYERS = 2
N_HEADS = 2
MAX_SEQ_LEN = 128

# Training
NUM_STEPS = 1000
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 0.0

# Optimizer (AdamW)
ADAM_BETA1 = 0.9
ADAM_BETA2 = 0.999
ADAM_EPSILON = 1e-8

# Initialization
INIT_SCALE = 0.02
SEED = 42

# Paths
TOKENIZER_PATH = "../data/flannel_tokenizer_chars.json"
CORPUS_PATH = "../data/flannel_model_corpus.txt"
TOKEN_MASK_PATH = "../tensors/Flannel/live_dead_tokens.safetensors"
OUTPUT_PATH = "../tensors/Thimble/thimble_3.safetensors"

print("✓ Parameters set")

✓ Parameters set


## Imports

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, GPT2LMHeadModel
from tokenizers import Tokenizer
import numpy as np
from pathlib import Path
from safetensors.torch import save_file, load_file
from tqdm.auto import tqdm
import time

print("✓ Imports complete")

✓ Imports complete


## Memory Safety Check

In [3]:
print(f"\n{'='*80}")
print(f"MEMORY & DISK SAFETY CHECK")
print(f"{'='*80}\n")

# Recording tensors - ALL BFLOAT16
bytes_bf16 = 2
bytes_f32 = 4

recording_w = (NUM_STEPS+1) * VOCAB_SIZE * HIDDEN_DIM * bytes_bf16
recording_grad = (NUM_STEPS+1) * VOCAB_SIZE * HIDDEN_DIM * bytes_bf16
recording_momentum = (NUM_STEPS+1) * VOCAB_SIZE * HIDDEN_DIM * bytes_bf16  # bfloat16!
recording_variance = (NUM_STEPS+1) * VOCAB_SIZE * HIDDEN_DIM * bytes_bf16  # bfloat16!
recording_losses = (NUM_STEPS+1) * bytes_f32

total_recording = recording_w + recording_grad + recording_momentum + recording_variance + recording_losses

print(f"Recording tensors (ALL BFLOAT16):")
print(f"  W:         {recording_w/1e9:.2f} GB (bfloat16)")
print(f"  grad_W:    {recording_grad/1e9:.2f} GB (bfloat16)")
print(f"  momentum:  {recording_momentum/1e9:.2f} GB (bfloat16) ← THE KEY CHANGE")
print(f"  variance:  {recording_variance/1e9:.2f} GB (bfloat16) ← THE KEY CHANGE")
print(f"  losses:    {recording_losses/1e9:.4f} GB (float32)")
print(f"  {'─'*40}")
print(f"  Total:     {total_recording/1e9:.2f} GB")
print()

print(f"This is EXACTLY what PyTorch uses internally.")
print(f"No dtype conversions. Pure bfloat16 pipeline.")
print(f"\n{'='*80}\n")


MEMORY & DISK SAFETY CHECK

Recording tensors (ALL BFLOAT16):
  W:         1.28 GB (bfloat16)
  grad_W:    1.28 GB (bfloat16)
  momentum:  1.28 GB (bfloat16) ← THE KEY CHANGE
  variance:  1.28 GB (bfloat16) ← THE KEY CHANGE
  losses:    0.0000 GB (float32)
  ────────────────────────────────────────
  Total:     5.13 GB

This is EXACTLY what PyTorch uses internally.
No dtype conversions. Pure bfloat16 pipeline.




## Device Detection

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

print(f"Using device: {device}")

Using device: mps


## Set Random Seeds

In [5]:
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"✓ Random seed set to {SEED}")

✓ Random seed set to 42


## Load Data

In [6]:
# Tokenizer
print(f"Loading tokenizer: {TOKENIZER_PATH}")
tokenizer = Tokenizer.from_file(str(TOKENIZER_PATH))
print(f"  ✓ Vocabulary: {tokenizer.get_vocab_size():,} tokens\n")

# Corpus
print(f"Loading corpus: {CORPUS_PATH}")
with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
    corpus_text = f.read()
encoding = tokenizer.encode(corpus_text)
tokens = encoding.ids
corpus_tensor = torch.tensor(tokens, dtype=torch.long)
print(f"  ✓ Tokens: {len(tokens):,}\n")

# Token masks (for analysis later)
print(f"Loading token masks: {TOKEN_MASK_PATH}")
mask_data = load_file(TOKEN_MASK_PATH)
live_mask = mask_data['live_mask'].bool()
dead_mask = mask_data['dead_mask'].bool()
n_live = live_mask.sum().item()
n_dead = dead_mask.sum().item()
print(f"  ✓ Live: {n_live:,} | Dead: {n_dead:,}")

Loading tokenizer: ../data/flannel_tokenizer_chars.json
  ✓ Vocabulary: 10,000 tokens

Loading corpus: ../data/flannel_model_corpus.txt
  ✓ Tokens: 1,371,328

Loading token masks: ../tensors/Flannel/live_dead_tokens.safetensors
  ✓ Live: 6,301 | Dead: 3,699


## Dataset and DataLoader

In [7]:
class TokenDataset(Dataset):
    def __init__(self, corpus_tensor, max_seq_len):
        self.corpus = corpus_tensor
        self.max_seq_len = max_seq_len
    
    def __len__(self):
        return max(0, len(self.corpus) - self.max_seq_len)
    
    def __getitem__(self, idx):
        chunk = self.corpus[idx : idx + self.max_seq_len + 1]
        return {
            'input_ids': chunk[:-1],
            'labels': chunk[1:]
        }

dataset = TokenDataset(corpus_tensor, MAX_SEQ_LEN)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(SEED)

dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    generator=g,
    worker_init_fn=seed_worker,
    num_workers=0,
)

print(f"\n✓ Dataset: {len(dataset):,} examples")
print(f"✓ DataLoader: {len(dataloader):,} batches per epoch")


✓ Dataset: 1,371,200 examples
✓ DataLoader: 10,713 batches per epoch


## Create Model (BFLOAT16)

In [8]:
print("Creating model...\n")

config = GPT2Config(
    vocab_size=VOCAB_SIZE,
    n_positions=MAX_SEQ_LEN,
    n_embd=HIDDEN_DIM,
    n_layer=N_LAYERS,
    n_head=N_HEADS,
    resid_pdrop=0.0,
    embd_pdrop=0.0,
    attn_pdrop=0.0,
    tie_word_embeddings=True,
)

model = GPT2LMHeadModel(config)

# Initialize embedding weights
with torch.no_grad():
    nn.init.normal_(model.transformer.wte.weight, mean=0.0, std=INIT_SCALE)

# Convert to bfloat16 and move to device
model = model.to(torch.bfloat16).to(device)

n_params = sum(p.numel() for p in model.parameters())

print(f"  Architecture: {N_LAYERS} layers, {N_HEADS} heads, {HIDDEN_DIM}d embeddings")
print(f"  Parameters: {n_params:,}")
print(f"  Device: {device}")
print(f"  Dtype: {model.transformer.wte.weight.dtype} (BFLOAT16)")
print(f"\n✓ Model created")

Creating model...

  Architecture: 2 layers, 2 heads, 64d embeddings
  Parameters: 748,288
  Device: mps
  Dtype: torch.bfloat16 (BFLOAT16)

✓ Model created


## Create Optimizer

In [9]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    betas=(ADAM_BETA1, ADAM_BETA2),
    eps=ADAM_EPSILON,
    weight_decay=WEIGHT_DECAY,
)

print(f"✓ Optimizer: AdamW")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Betas: ({ADAM_BETA1}, {ADAM_BETA2})")
print(f"  Epsilon: {ADAM_EPSILON}")
print(f"  Weight decay: {WEIGHT_DECAY}")
print(f"\n  Optimizer states will be BFLOAT16 (matching param dtype)")

✓ Optimizer: AdamW
  Learning rate: 0.001
  Betas: (0.9, 0.999)
  Epsilon: 1e-08
  Weight decay: 0.0

  Optimizer states will be BFLOAT16 (matching param dtype)


## Pre-allocate Recording Tensors (ALL BFLOAT16)

In [10]:
print("\nPre-allocating recording tensors...\n")

# ALL BFLOAT16 - exactly what PyTorch uses
W_history = torch.zeros(NUM_STEPS+1, VOCAB_SIZE, HIDDEN_DIM, dtype=torch.bfloat16)
grad_history = torch.zeros(NUM_STEPS+1, VOCAB_SIZE, HIDDEN_DIM, dtype=torch.bfloat16)
momentum_history = torch.zeros(NUM_STEPS+1, VOCAB_SIZE, HIDDEN_DIM, dtype=torch.bfloat16)
variance_history = torch.zeros(NUM_STEPS+1, VOCAB_SIZE, HIDDEN_DIM, dtype=torch.bfloat16)
loss_history = torch.zeros(NUM_STEPS+1, dtype=torch.float32)

# Memory calculation
bytes_per_bf16 = 2
memory_w = W_history.numel() * bytes_per_bf16
memory_grad = grad_history.numel() * bytes_per_bf16
memory_momentum = momentum_history.numel() * bytes_per_bf16
memory_variance = variance_history.numel() * bytes_per_bf16
memory_loss = loss_history.numel() * 4
total_memory = memory_w + memory_grad + memory_momentum + memory_variance + memory_loss

print(f"  W:         {tuple(W_history.shape)} (bfloat16) = {memory_w/1e9:.2f} GB")
print(f"  grad_W:    {tuple(grad_history.shape)} (bfloat16) = {memory_grad/1e9:.2f} GB")
print(f"  momentum:  {tuple(momentum_history.shape)} (bfloat16) = {memory_momentum/1e9:.2f} GB")
print(f"  variance:  {tuple(variance_history.shape)} (bfloat16) = {memory_variance/1e9:.2f} GB")
print(f"  losses:    {tuple(loss_history.shape)} (float32) = {memory_loss/1e9:.4f} GB")
print(f"\n  Total: {total_memory/1e9:.2f} GB")
print(f"\n✓ Tensors allocated (ALL BFLOAT16 - matching PyTorch's internal precision)")


Pre-allocating recording tensors...

  W:         (1001, 10000, 64) (bfloat16) = 1.28 GB
  grad_W:    (1001, 10000, 64) (bfloat16) = 1.28 GB
  momentum:  (1001, 10000, 64) (bfloat16) = 1.28 GB
  variance:  (1001, 10000, 64) (bfloat16) = 1.28 GB
  losses:    (1001,) (float32) = 0.0000 GB

  Total: 5.13 GB

✓ Tensors allocated (ALL BFLOAT16 - matching PyTorch's internal precision)


## Training Loop

In [11]:
print(f"\n{'='*80}")
print(f"THIMBLE 3: THE ALL-BFLOAT16 REVUE 🎭")
print(f"{'='*80}\n")

# Record initial state (step 0)
W_history[0] = model.transformer.wte.weight.data.clone().cpu()
loss_history[0] = float('nan')
print("✓ Recorded initial state (t=0)\n")

# Create infinite iterator over dataloader
data_iter = iter(dataloader)

# Training loop
model.train()
start_time = time.time()

for step in tqdm(range(1, NUM_STEPS+1), desc="Training"):
    # Get next batch
    try:
        batch = next(data_iter)
    except StopIteration:
        data_iter = iter(dataloader)
        batch = next(data_iter)
    
    # Move batch to device
    input_ids = batch['input_ids'].to(device)
    labels = batch['labels'].to(device)
    
    # Forward pass
    outputs = model(input_ids=input_ids, labels=labels)
    loss = outputs.loss
    
    # Backward pass
    loss.backward()
    
    # === RECORD GRADIENTS (before optimizer.step) ===
    grad_history[step] = model.transformer.wte.weight.grad.clone().cpu()
    
    # Optimizer step
    optimizer.step()
    optimizer.zero_grad()
    
    # === RECORD WEIGHTS & OPTIMIZER STATE (after optimizer.step) ===
    # NO DTYPE CONVERSION - record exactly what PyTorch has
    W_history[step] = model.transformer.wte.weight.data.clone().cpu()
    
    wte_param = model.transformer.wte.weight
    if wte_param in optimizer.state:
        opt_state = optimizer.state[wte_param]
        # Record as-is (bfloat16)
        momentum_history[step] = opt_state['exp_avg'].clone().cpu()
        variance_history[step] = opt_state['exp_avg_sq'].clone().cpu()
    
    loss_history[step] = loss.item()

elapsed = time.time() - start_time

print(f"\n{'='*80}")
print(f"✓ Training complete")
print(f"  Time: {elapsed:.1f}s ({elapsed/60:.1f} minutes)")
print(f"  Final loss: {loss_history[-1]:.4f}")
print(f"{'='*80}")


THIMBLE 3: THE ALL-BFLOAT16 REVUE 🎭

✓ Recorded initial state (t=0)



Training:   0%|          | 0/1000 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.



✓ Training complete
  Time: 75.7s (1.3 minutes)
  Final loss: 6.5621


## Validation: Predict W[t] Using Bfloat16 Pipeline

Now we simulate EXACTLY what PyTorch does, quantizing at every step.

In [12]:
print("\nValidating bfloat16 accounting...\n")

test_steps = [1, 10, 50, 100, 200, 500, 800]

for t in test_steps:
    if t > NUM_STEPS:
        continue
    
    # Get bfloat16 optimizer states (as recorded - no conversion)
    m_t = momentum_history[t]  # bfloat16
    v_t = variance_history[t]  # bfloat16
    W_prev = W_history[t-1]    # bfloat16
    
    # Simulate PyTorch's bfloat16 pipeline
    # (We could do this in float32 then quantize, or just use bfloat16 throughout)
    bias_correction1 = 1 - ADAM_BETA1**t
    bias_correction2 = 1 - ADAM_BETA2**t
    
    # All in bfloat16
    m_hat = (m_t.float() / bias_correction1).bfloat16()
    v_hat = (v_t.float() / bias_correction2).bfloat16()
    dW = (-LEARNING_RATE * m_hat.float() / (torch.sqrt(v_hat.float()) + ADAM_EPSILON)).bfloat16()
    predicted_W = (W_prev.float() + dW.float()).bfloat16()
    
    # Observed W[t]
    observed_W = W_history[t]
    
    # Test for exact equality (or very close)
    exactly_equal = torch.equal(predicted_W, observed_W)
    max_abs_diff = torch.max(torch.abs(predicted_W.float() - observed_W.float())).item()
    
    # Approximate metrics
    norm_pred = torch.norm((predicted_W - W_prev).float())
    norm_obs = torch.norm((observed_W - W_prev).float())
    ratio = norm_pred / (norm_obs + 1e-10)
    
    print(f"t={t:3d}: exact_match={exactly_equal}, max_diff={max_abs_diff:.2e}, ratio={ratio:.6f}")

print("\nIf exact_match=True (or max_diff << bfloat16 precision), we've matched PyTorch!")
print("\n✓ Validation complete")


Validating bfloat16 accounting...

t=  1: exact_match=False, max_diff=2.44e-04, ratio=0.999207
t= 10: exact_match=False, max_diff=4.88e-04, ratio=1.000738
t= 50: exact_match=False, max_diff=4.88e-04, ratio=0.998727
t=100: exact_match=False, max_diff=4.88e-04, ratio=0.999764
t=200: exact_match=False, max_diff=9.77e-04, ratio=1.000013
t=500: exact_match=False, max_diff=1.95e-03, ratio=0.986503
t=800: exact_match=False, max_diff=9.77e-04, ratio=0.999655

If exact_match=True (or max_diff << bfloat16 precision), we've matched PyTorch!

✓ Validation complete


## Save Data

In [13]:
print(f"\nSaving data to {OUTPUT_PATH}...\n")

Path(OUTPUT_PATH).parent.mkdir(parents=True, exist_ok=True)

save_dict = {
    # Training trajectories (ALL BFLOAT16)
    'W': W_history,
    'grad_W': grad_history,
    'momentum_W': momentum_history,
    'variance_W': variance_history,
    'losses': loss_history,
    
    # Hyperparameters
    'vocab_size': torch.tensor(VOCAB_SIZE, dtype=torch.long),
    'hidden_dim': torch.tensor(HIDDEN_DIM, dtype=torch.long),
    'n_layers': torch.tensor(N_LAYERS, dtype=torch.long),
    'n_heads': torch.tensor(N_HEADS, dtype=torch.long),
    'num_steps': torch.tensor(NUM_STEPS, dtype=torch.long),
    'batch_size': torch.tensor(BATCH_SIZE, dtype=torch.long),
    'learning_rate': torch.tensor(LEARNING_RATE, dtype=torch.float32),
    'weight_decay': torch.tensor(WEIGHT_DECAY, dtype=torch.float32),
    'adam_beta1': torch.tensor(ADAM_BETA1, dtype=torch.float32),
    'adam_beta2': torch.tensor(ADAM_BETA2, dtype=torch.float32),
    'adam_epsilon': torch.tensor(ADAM_EPSILON, dtype=torch.float32),
    'init_scale': torch.tensor(INIT_SCALE, dtype=torch.float32),
    'seed': torch.tensor(SEED, dtype=torch.long),
    'n_live': torch.tensor(n_live, dtype=torch.long),
    'n_dead': torch.tensor(n_dead, dtype=torch.long),
}

save_start = time.time()
save_file(save_dict, str(OUTPUT_PATH))
save_elapsed = time.time() - save_start

file_size_bytes = Path(OUTPUT_PATH).stat().st_size
file_size_gb = file_size_bytes / 1e9

print(f"✓ Saved successfully")
print(f"  File: {Path(OUTPUT_PATH).name}")
print(f"  Size: {file_size_gb:.2f} GB")
print(f"  Save time: {save_elapsed:.1f}s")


Saving data to ../tensors/Thimble/thimble_3.safetensors...

✓ Saved successfully
  File: thimble_3.safetensors
  Size: 5.13 GB
  Save time: 2.3s


## Summary

In [14]:
print(f"\n{'='*80}")
print(f"THIMBLE 3 COMPLETE: THE ALL-BFLOAT16 REVUE 🎭")
print(f"{'='*80}\n")

print(f"Trained for {NUM_STEPS:,} steps with PURE BFLOAT16 pipeline")
print(f"  Model dtype: bfloat16")
print(f"  Optimizer states: bfloat16 (matching PyTorch internal precision)")
print(f"  No dtype conversions during recording")
print()
print(f"Recorded at every step (all bfloat16):")
print(f"  • W: embedding weights")
print(f"  • grad_W: gradients")
print(f"  • momentum_W: Adam exp_avg")
print(f"  • variance_W: Adam exp_avg_sq")
print(f"  • losses: training loss")
print()
print(f"Data saved: {OUTPUT_PATH}")
print(f"  Size: {file_size_gb:.2f} GB")
print(f"  Training time: {elapsed/60:.1f} minutes")
print()
print(f"This is EXACTLY what PyTorch does. Pure bfloat16. Quantization at every step.")
print(f"If the validation showed exact matches, we've cracked it. 🎉")
print(f"\n{'='*80}")


THIMBLE 3 COMPLETE: THE ALL-BFLOAT16 REVUE 🎭

Trained for 1,000 steps with PURE BFLOAT16 pipeline
  Model dtype: bfloat16
  Optimizer states: bfloat16 (matching PyTorch internal precision)
  No dtype conversions during recording

Recorded at every step (all bfloat16):
  • W: embedding weights
  • grad_W: gradients
  • momentum_W: Adam exp_avg
  • variance_W: Adam exp_avg_sq
  • losses: training loss

Data saved: ../tensors/Thimble/thimble_3.safetensors
  Size: 5.13 GB
  Training time: 1.3 minutes

This is EXACTLY what PyTorch does. Pure bfloat16. Quantization at every step.
If the validation showed exact matches, we've cracked it. 🎉

